In [2]:
pip3 install gdal

SyntaxError: invalid syntax (3786297673.py, line 1)

In [16]:
pip install pingouin

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 204 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 16.5 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import scipy.stats as stats
import requests
import rasterio
from rasterio.plot import show
import richdem as rd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.stats import poisson
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
# import gdal
#from osgeo import gdal


In [23]:
# Download a DEM 

# Your OpenTopography API key
OPENTOPO_API_KEY = "f95f74790401d94ee1989cd4cffea1d7"

# Define the bounding box for the area of interest
min_lon, min_lat = -109.0, 37.0
max_lon, max_lat = -102.0, 41.0

# OpenTopography API URL
url = "https://portal.opentopography.org/API/globaldem"

# Parameters for the API request
params = {
    'demtype': 'SRTMGL1',  # You can choose other DEM types like 'SRTMGL3', 'AW3D30', etc.
    'south': min_lat,
    'north': max_lat,
    'west': min_lon,
    'east': max_lon,
    'outputFormat': 'GTiff',
    'API_Key': OPENTOPO_API_KEY
}

# Make the API request
response = requests.get(url, params=params)

# Save the DEM file
dem_path = 'dem2.tif'
with open(dem_path, 'wb') as f:
    f.write(response.content)

with rasterio.open(dem_path) as src:
    # Calculate the window to read based on the desired coordinates
    window = rasterio.windows.from_bounds(min_lon, min_lat, max_lon, max_lat, transform=src.transform)
    dem = src.read(1, window=window)
    transform = src.window_transform(window)


In [25]:
dem

array([[2544, 2542, 2539, ..., 1097, 1096, 1096],
       [2546, 2545, 2542, ..., 1097, 1096, 1096],
       [2544, 2545, 2544, ..., 1097, 1097, 1098],
       ...,
       [1424, 1424, 1423, ..., 1115, 1113, 1113],
       [1423, 1424, 1424, ..., 1114, 1114, 1112],
       [1422, 1424, 1424, ..., 1115, 1114, 1114]], dtype=int16)

In [64]:
dem = rd.LoadGDAL('dem2.tif')

Exception: richdem.LoadGDAL() requires GDAL.

In [3]:
precip_data = xr.open_dataset("../Data/Colorado_accum.nc")
precip_data
inst_data = xr.open_dataset("../Data/Colorado_instant.nc")
inst_data

<xarray.Dataset> Size: 11MB
Dimensions:     (valid_time: 672, latitude: 29, longitude: 45)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 5kB 2025-02-01 ... 2025-02-28T23:...
  * latitude    (latitude) float64 232B 42.5 42.25 42.0 ... 36.0 35.75 35.5
  * longitude   (longitude) float64 360B -111.5 -111.2 -111.0 ... -100.8 -100.5
    expver      (valid_time) <U4 11kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 4MB ...
    v10         (valid_time, latitude, longitude) float32 4MB ...
    t2m         (valid_time, latitude, longitude) float32 4MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-04-18T22:43 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
avalanche_data = pd.read_csv("../Data/avalanche_colorado.csv")
avalanche_data['Date'] = avalanche_data['Date'].str.split(' GMT').str[0]
avalanche_data['Date'] = pd.to_datetime(avalanche_data['Date'])
avalanche_data['Date'] = avalanche_data['Date'].dt.strftime('%b %d')
avalanche_data = avalanche_data.loc[:, ["Date", "Longitude", "latitude"]]
avalanche_data = avalanche_data[avalanche_data['Date'] == 'Feb 25']
avalanche_data

,Date,Longitude,latitude
41,Feb 25,-106.926922,39.168065
42,Feb 25,-107.792203,37.908853
43,Feb 25,-107.843197,37.853397
44,Feb 25,-106.388902,39.022677
45,Feb 25,-105.724746,39.599387
46,Feb 25,-106.180301,39.463320
47,Feb 25,-108.039188,37.449961
48,Feb 25,-106.558147,39.082909
49,Feb 25,-106.558147,39.082909
50,Feb 25,-105.869044,39.833220


In [5]:
lon_min = precip_data['longitude'].min().item()
lon_max = precip_data['longitude'].max().item()
lat_min = precip_data['latitude'].min().item()
lat_max = precip_data['latitude'].max().item()

lon_edges = np.arange(lon_max, lon_min - 0.25, -0.25)
lat_edges = np.arange(lat_min, lat_max + 0.25, 0.25)

lon_edges


array([-100.5 , -100.75, -101.  , -101.25, -101.5 , -101.75, -102.  ,
       -102.25, -102.5 , -102.75, -103.  , -103.25, -103.5 , -103.75,
       -104.  , -104.25, -104.5 , -104.75, -105.  , -105.25, -105.5 ,
       -105.75, -106.  , -106.25, -106.5 , -106.75, -107.  , -107.25,
       -107.5 , -107.75, -108.  , -108.25, -108.5 , -108.75, -109.  ,
       -109.25, -109.5 , -109.75, -110.  , -110.25, -110.5 , -110.75,
       -111.  , -111.25, -111.5 ])

In [13]:
avalanche_count = []
precip_by_cell = []
rad_by_cell = []
temp_by_cell = []
u10_by_cell = []
v10_by_cell = []

for i in range(len(lon_edges) - 1):
    for j in range(len(lat_edges) - 1):
        count = len(avalanche_data[((avalanche_data['latitude'] >= lat_edges[j]) & (avalanche_data['latitude'] <= lat_edges[j+1])) &
                                   ((avalanche_data['Longitude'] <= lon_edges[i]) & (avalanche_data['Longitude'] >= lon_edges[i+1]))])
        avalanche_count.append(count)
        precip_by_cell.append(precip_data['tp'].sel(valid_time='2025-02-25', latitude=lat_edges[j], longitude=lon_edges[i], method='nearest').item())
        rad_by_cell.append(precip_data['ssrd'].sel(valid_time='2025-02-25', latitude=lat_edges[j], longitude=lon_edges[i], method='nearest').item())
        temp_by_cell.append(inst_data['t2m'].sel(valid_time='2025-02-25', latitude=lat_edges[j], longitude=lon_edges[i], method='nearest').item())
        u10_by_cell.append(inst_data['u10'].sel(valid_time='2025-02-25', latitude=lat_edges[j], longitude=lon_edges[i], method='nearest').item())
        v10_by_cell.append(inst_data['v10'].sel(valid_time='2025-02-25', latitude=lat_edges[j], longitude=lon_edges[i], method='nearest').item())

In [7]:
df = pd.DataFrame({'avalanche_count': avalanche_count, 'precip': precip_by_cell, 'rad': rad_by_cell, 'temp': temp_by_cell, 'u10': u10_by_cell, 'v10': v10_by_cell})
df['rad'] /= 1000000
#df['temp'] = (df['temp'] - 272.15) * 1.8 + 32 # Convert Kelvin to Fahrenheit
df

,avalanche_count,precip,rad,temp,u10,v10
0,0,0.0,0.588608,291.786377,-1.314651,-2.487442
1,0,0.0,0.584192,290.860596,-1.636917,-2.184708
2,0,0.0,0.578752,290.764893,-2.111526,-1.845840
3,0,0.0,0.572544,290.614502,-2.369339,-1.544083
4,0,0.0,0.564352,290.901611,-2.572464,-1.419083
...,...,...,...,...,...,...
1227,0,0.0,1.049216,280.706299,1.046677,1.882675
1228,0,0.0,1.053696,280.094971,1.568161,1.592636
1229,0,0.0,1.040896,278.509033,1.956833,1.345566
1230,0,0.0,1.005056,280.848877,0.937302,0.408066


In [15]:
def model_parameters(reg, columns):
    """Returns a string with the linear regression model parameters for the given column names."""
    slopes = [f"{coef:.2f}({columns[i]})" for i, coef in enumerate(reg.coef_)]
    return " + ".join([f"{reg.intercept_:.2f}"] + slopes)

predictors = ['precip', 'rad', 'temp', 'u10', 'v10']  
outcome = 'avalanche_count'

X = df[predictors]
y = df[outcome]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  # 80% training, 20% testing

model = LinearRegression().fit(X_train, y_train)
#model = LinearRegression().fit(df.loc[:, ['precip', 'rad', 'temp', 'u10', 'v10']], df['avalanche_count'])

print("Model:", model_parameters(model, X_train.columns))
print("Error:", root_mean_squared_error(y_test, model.predict(X_test)))

print("Model Coefficients:")
for feature, coef in zip(predictors, model.coef_):
    print(f"{feature:10s}: {coef:.4f}") 

Model: 0.87 + -46154.41(precip) + 0.00(rad) + -0.00(temp) + -0.00(u10) + -0.00(v10)
Error: 0.16463382752693653
Model Coefficients:
precip    : -46154.4093
rad       : 0.0045
temp      : -0.0030
u10       : -0.0013
v10       : -0.0041


In [29]:
for predict in predictors:
    print("\n" + str(predict) + ": ")
    print(pg.corr(df[predict], df['avalanche_count']))
#pg.corr(df['precip'], df['avalanche_count'])


precip: 
            n         r          CI95%     p-val   BF10     power
pearson  1232 -0.008318  [-0.06, 0.05]  0.770531  0.037  0.059792

rad: 
            n         r          CI95%     p-val   BF10     power
pearson  1232  0.037048  [-0.02, 0.09]  0.193773  0.083  0.255131

temp: 
            n         r          CI95%         p-val       BF10     power
pearson  1232 -0.143989  [-0.2, -0.09]  3.868299e-07  1.378e+04  0.999111

u10: 
            n         r          CI95%     p-val  BF10     power
pearson  1232  0.032995  [-0.02, 0.09]  0.247171  0.07  0.212047

v10: 
            n         r          CI95%     p-val   BF10     power
pearson  1232 -0.008331  [-0.06, 0.05]  0.770194  0.037  0.059821


In [14]:
model = sm.GLM(y_train, X_train, family=sm.families.Poisson())
results = model.fit()

print(results.summary())
print(results.pvalues)


                 Generalized Linear Model Regression Results                  
Dep. Variable:        avalanche_count   No. Observations:                  985
Model:                            GLM   Df Residuals:                      980
Model Family:                 Poisson   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -52.310
Date:                Thu, 24 Apr 2025   Deviance:                       87.392
Time:                        10:52:15   Pearson chi2:                     838.
No. Iterations:                    18   Pseudo R-squ. (CS):            0.01006
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
precip     -3.662e+07   3.49e+10     -0.001      0.9